# Manejo de marcas de tiempo faltantes

En esta lección veremos las técnicas que podemos usar para el manejo de marcas de tiempo faltantes, es decir cuando la variable temporal de nuestra Serie de Tiempo contiene valores faltantes.

El manejo de las marcas de tiempo faltantes **es el primer paso que siempre debemos llevar a cabo** antes de, por ejemplo, realizar el manejo de observaciones faltantes. Esto se debe a que sin la variable temporal completa no podremos posteriormente realizar las demás tareas de pre-procesamiento de datos.

Además, es importante tener en cuenta que para realizar un manejo adecuado de datos faltantes (sean marcas de tiempo u observaciones) **es esencial tener un conocimiento detallado de los datos que estamos procesando**. Sin este conocimiento no sabremos qué decisión tomar en cada caso particular al momento de realizar el manejo.

## 1. Manejo de marcas de tiempo faltantes

Recordemos que en este caso las marcas de tiempo incompletas serán marcadas como *NaT*.

El manejo de este tipo de marcas de tiempo faltantes dependerá de si hay o no correspondencia entre las marcas faltantes y la frecuencia de la serie.

### 1.1. Caso 1: NO hay correspondencia entre las marcas faltantes y la frecuencia de la serie

Supongamos que tenemos esta Serie de Tiempo:

|   Índice   | Observación |
|:----------:|:-----------:|
| 2020-01-01 | 0.25        |
| 2020-01-02 | 3.27        |
| NaT        | 0.9         |
| NaT        | 4.3         |
| NaT        | 1.3         |
| 2020-01-04 | 2.92        |

En este caso vemos que la Serie tiene una frecuencia diaria y que al parecer falta la marca correspondiente al 2020-01-03. Sin embargo, tenemos 3 observaciones que podrían pertenecer a esa marca.

En este caso podemos eliminar los registros consecutivos con "NaT" y crear uno nuevo con la(s) fecha(s) faltantes. En este caso aparecerá una observación tipo "NaN" que deberá ser manejada usando alguna de las técnicas que veremos más adelante.

Veamos en detalle esta situación:

In [1]:
import pandas as pd

# Crear el DataFrame con múltiples marcas faltantes
ind = ['2020-01-01', '2020-01-02', 'NaT', 'NaT', 'NaT', '2020-01-04']
obs = [0.25, 3.27, 0.9, 4.3, 1.3, 2.92]
ind = pd.to_datetime(ind)
df = pd.DataFrame(obs, index=ind, columns=['Observación'])
df

,Observación
2020-01-01,0.25
2020-01-02,3.27
NaT,0.90
NaT,4.30
NaT,1.30
2020-01-04,2.92


In [2]:
# Eliminar registros cuyo índice sea "NaT"
# Esto equivale a preservar las filas cuyo índice NO SEA "NaT"
df_clean = df[pd.notna(df.index)]
df_clean

,Observación
2020-01-01,0.25
2020-01-02,3.27
2020-01-04,2.92


In [3]:
# Reindexar

# Ordenar el DataFrame original de manera ascendente según su índice (por precaución)
df_clean = df_clean.sort_index()

# Definir rango completo
rango_completo = pd.date_range(start=df_clean.index.min(), end=df_clean.index.max(), freq='D')
print(rango_completo)

# Reindexar con base en el rango completo
df_clean = df_clean.reindex(rango_completo)

print('DataFrame original:')
print(df)
print('='*30)
print('DataFrame pre-procesado:')
print(df_clean)

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'], dtype='datetime64[ns]', freq='D')
DataFrame original:
            Observación
2020-01-01         0.25
2020-01-02         3.27
NaT                0.90
NaT                4.30
NaT                1.30
2020-01-04         2.92
DataFrame pre-procesado:
            Observación
2020-01-01         0.25
2020-01-02         3.27
2020-01-03          NaN
2020-01-04         2.92


### 1.2. Caso 2: hay correspondencia entre las marcas faltantes y la frecuencia de la serie

Supongamos que tenemos esta Serie de Tiempo:

|   Índice   | Observación |
|:----------:|:-----------:|
| 2020-01-01 | 0.25        |
| 2020-01-02 | 3.27        |
| NaT        | 0.9         |
| 2020-01-04 | 4.3         |
| NaT        | 1.3         |
| 2020-01-06 | 2.92        |

En este caso vemos que la Serie tiene una frecuencia diaria y que al parecer faltan las marcas correspondiente al 2020-01-03 y 2020-01-05, para cada una de las cuales tenemos exactamente 1 observación.

En este caso podemos:

1. Definir un nuevo índice con el rango específico de fechas (inicio-final)
2. Fijar este nuevo índice como índice del *DataFrame* existente

Veamos este ejemplo:

In [4]:
# Crear el DataFrame con marcas faltantes
ind = ['2020-01-01', '2020-01-02', 'NaT', '2020-01-04', 'NaT', '2020-01-06']
obs = [0.25, 3.27, 0.9, 4.3, 1.3, 2.92]
ind = pd.to_datetime(ind)
df = pd.DataFrame(obs, index=ind, columns=['Observación'])
df

,Observación
2020-01-01,0.25
2020-01-02,3.27
NaT,0.90
2020-01-04,4.30
NaT,1.30
2020-01-06,2.92


In [5]:
# 1. Crear un rango completo de fechas
rango_completo = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
rango_completo

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06'],
              dtype='datetime64[ns]', freq='D')

In [6]:
# 2. Fijar el nuevo índice como índice del DataFrame existente
df_clean = df.set_index(rango_completo)

print('Dataframe original: ')
print(df)
print('='*30)
print('Dataframe pre-procesado: ')
print(df_clean)

Dataframe original: 
            Observación
2020-01-01         0.25
2020-01-02         3.27
NaT                0.90
2020-01-04         4.30
NaT                1.30
2020-01-06         2.92
Dataframe pre-procesado: 
            Observación
2020-01-01         0.25
2020-01-02         3.27
2020-01-03         0.90
2020-01-04         4.30
2020-01-05         1.30
2020-01-06         2.92


## 2. Manejo de marcas de tiempo faltantes "ocultas"

Recordemos que en este caso ninguna marca de tiempo aparece etiquetada como *NaT* y sin embargo tendremos marcas faltantes.

Por ejemplo el *DataFrame*:

|   Índice   | Observación |
|:----------:|:-----------:|
| 2020-01-01 | 0.25        |
| 2020-01-02 | 3.27        |
| 2020-01-04 | 2.92        |

No contiene *NaT*s en su índice y sin embargo falta el registro correspondiente a la fecha 2022-01-03.

En este caso podemos reindexar la Serie de Tiempo teniendo en cuenta que **se agregarán tantos NaN a las observaciones como elementos se añadan al índice original**.

Veamos esta implementación:

In [7]:
# Crear el DataFrame con marcas faltantes
ind = ['2020-01-01', '2020-01-02', '2020-01-04']
obs = [0.25, 3.27, 2.92]
ind = pd.to_datetime(ind)
df = pd.DataFrame(obs, index=ind, columns=['Observación'])
df

,Observación
2020-01-01,0.25
2020-01-02,3.27
2020-01-04,2.92


In [8]:
# Ordenar la serie de manera ascendente según su índice (por precaución)
df_clean = df.sort_index()

# Crear nuevo índice
rango_completo = pd.date_range(start=df_clean.index.min(), end=df_clean.index.max(), freq='D')

# Reindexar
df_clean = df_clean.reindex(rango_completo)

print('Dataframe original: ')
print(df)
print('='*30)
print('Dataframe pre-procesado: ')
print(df_clean)

Dataframe original: 
            Observación
2020-01-01         0.25
2020-01-02         3.27
2020-01-04         2.92
Dataframe pre-procesado: 
            Observación
2020-01-01         0.25
2020-01-02         3.27
2020-01-03          NaN
2020-01-04         2.92


Vemos que en el caso anterior tan sólo había una marca faltante entre las fechas 2020-01-02 y 2020-01-04 y por tanto se agregó tan sólo una observación (NaN) en la nueva Serie de Tiempo.

Sin embargo, **este método no es aconsejable cuando tenemos bloques de tiempo demasiado grandes en comparación con la frecuencia de la Serie**.

Por ejemplo, si ahora la Serie de Tiempo es de la forma:

|   Índice   | Observación |
|:----------:|:-----------:|
| 2020-01-01 | 0.25        |
| 2020-01-02 | 3.27        |
| 2020-01-08 | 2.92        |

Vemos que entre 2020-01-02 y 2020-01-08 ahora tendremos 5 marcas de tiempo faltantes. Por tanto, si usamos el método anterior añadiremos 5 observaciones tipo NaN que más adelante tendremos que manejar.

Esto no es recomendable porque tendremos un exceso de valores NaN por manejar para los cuales no tenemos suficiente información que nos permita completarlos.

En este caso se sugiere adquirir nuevamente los datos (de ser posible).

## 3. Ejemplo práctico

> Realizar el manejo de marcas de tiempo faltantes en el set de datos *clicks_faltantes_marcas.csv*

Comencemos leyendo el set de datos:

In [9]:
# Leer dataset
RUTA = '../datasets/missing_values/'

df = pd.read_csv(RUTA + 'clicks_faltantes_marcas.csv',
                 parse_dates = ['fecha'],
                 index_col = ['fecha'])
df

C:\Users\ASUS TUF\AppData\Local\Temp\ipykernel_25928\1401911102.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(RUTA + 'clicks_faltantes_marcas.csv',


,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2,18784.0
2008-04-02,43.079056,1,24738.0
NaT,43.842609,2,15209.0
2008-04-04,43.382794,2,14320.0
2008-04-05,43.941176,1,11974.0
...,...,...,...
2008-08-09,44.182033,1,6716.0
2008-08-10,43.608260,1,9523.0
2008-08-11,43.553363,1,8881.0


Y en primer lugar determinemos cuántas marcas *NaT* tenemos:

In [10]:
df.index.isna().sum()

2

Veamos dónde deberían estar ubicadas idealmente:

In [11]:
# DataFrame sin marcas NaT ordenado de manera ascendente
df_sin_nat = df[pd.notna(df.index)].sort_index()

# Rango de fechas ideal
rango_fechas = pd.date_range(start=df_sin_nat.index.min(), end=df_sin_nat.index.max(), freq='D')

# Diferencia entre el rango de fechas ideal y el índice de la Serie de Tiempo
rango_fechas.difference(df_sin_nat.index)

DatetimeIndex(['2008-04-03', '2008-04-07', '2008-04-09'], dtype='datetime64[ns]', freq=None)

Vemos que en realidad son 3 las marcas de tiempo faltantes, aunque el uso de `clicks_df.index.isna().sum()` nos indicaba que eran 2.

Veamos en detalle esta porción de la Serie de Tiempo:

In [12]:
df.iloc[0:10,:]

,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2,18784.0
2008-04-02,43.079056,1,24738.0
NaT,43.842609,2,15209.0
2008-04-04,43.382794,2,14320.0
2008-04-05,43.941176,1,11974.0
2008-04-06,44.403936,1,11007.0
NaT,43.995888,2,15214.0
2008-04-08,43.373773,1,11333.0
2008-04-10,43.154738,5,15677.0


En este caso vemos que las marcas correspondientes a 2008-04-03 y 2008-04-07 están marcadas como *NaT*. Sin embargo hay una tercera marca faltante que está oculta y que corresponde a 2008-04-09.

Es decir que tenemos dos situaciones en este caso:

- 2 marcas faltantes tipo "NaT"
- 1 marca faltante "oculta"

Posibles soluciones:

1. Si intentamos reindexar la serie con el rango de fechas ideal aparecerá un error pues tendremos 2 marcas "NaT"
2. Si intentamos fijar el índice de la serie como el rango de fechas ideal aparecerá un error pues este rango ideal tendrá 1 registro adicional (correspondiente a la marca oculta faltante)

Así que en este caso podríamos:

1. Crear el índice ideal **eliminando la fecha correspondiente a 2008-04-09 (marca oculta)**
2. Usar el resultado de (1) para fijar el nuevo índice del *DataFrame*. En este punto ya habremos completado las marcas "NaT"
3. Crear el índice ideal incluyendo la marca oculta (2008-04-09) y reindexar el *DataFrame* obtenido en (2). En este punto habremos completado la marca oculta y habremos agregado tres observación *NaN* en la fila correspondiente.

Veamos esta implementación:

In [13]:
# 1. Crear el índice ideal eliminando la fecha correspondiente a 2008-04-09 (marca oculta)

ind_ideal = pd.date_range(start=df_sin_nat.index.min(), end=df_sin_nat.index.max(), freq='D')
ind_filtrado = ind_ideal[ind_ideal != '2008-04-09']
ind_filtrado

DatetimeIndex(['2008-04-01', '2008-04-02', '2008-04-03', '2008-04-04',
               '2008-04-05', '2008-04-06', '2008-04-07', '2008-04-08',
               '2008-04-10', '2008-04-11',
               ...
               '2008-08-04', '2008-08-05', '2008-08-06', '2008-08-07',
               '2008-08-08', '2008-08-09', '2008-08-10', '2008-08-11',
               '2008-08-12', '2008-08-13'],
              dtype='datetime64[ns]', length=134, freq=None)

In [14]:
# 2. Usar el resultado de (1) para fijar el nuevo índice del DataFrame.
# En este punto ya habremos completado las marcas "NaT"

df_clean = df.set_index(ind_filtrado)

print('Dataframe original (posiciones 0:10):')
print(df.iloc[0:10,:])
print('='*30)
print('Dataframe preprocesado (posiciones 0:10):')
print(df_clean.iloc[0:10,:])

Dataframe original (posiciones 0:10):
               precio  ubicación   clicks
fecha                                    
2008-04-01  43.155647          2  18784.0
2008-04-02  43.079056          1  24738.0
NaT         43.842609          2  15209.0
2008-04-04  43.382794          2  14320.0
2008-04-05  43.941176          1  11974.0
2008-04-06  44.403936          1  11007.0
NaT         43.995888          2  15214.0
2008-04-08  43.373773          1  11333.0
2008-04-10  43.154738          5  15677.0
2008-04-11  42.921659          2  10792.0
Dataframe preprocesado (posiciones 0:10):
               precio  ubicación   clicks
2008-04-01  43.155647          2  18784.0
2008-04-02  43.079056          1  24738.0
2008-04-03  43.842609          2  15209.0
2008-04-04  43.382794          2  14320.0
2008-04-05  43.941176          1  11974.0
2008-04-06  44.403936          1  11007.0
2008-04-07  43.995888          2  15214.0
2008-04-08  43.373773          1  11333.0
2008-04-10  43.154738          5  1567

In [15]:
# 3. Crear el índice ideal incluyendo la marca oculta (2008-04-09) y reindexar
# el DataFrame obtenido en (2). En este punto habremos completado la marca oculta
# y habremos agregado tres observaciones NaN en la fila correspondiente.

# El índice ideal está en el arreglo "ind_ideal". Reindexación:
df_clean = df_clean.reindex(ind_ideal)

print('Dataframe original (posiciones 0:10):')
print(df.iloc[0:10,:])
print('='*30)
print('Dataframe preprocesado (posiciones 0:10):')
print(df_clean.iloc[0:10,:])

Dataframe original (posiciones 0:10):
               precio  ubicación   clicks
fecha                                    
2008-04-01  43.155647          2  18784.0
2008-04-02  43.079056          1  24738.0
NaT         43.842609          2  15209.0
2008-04-04  43.382794          2  14320.0
2008-04-05  43.941176          1  11974.0
2008-04-06  44.403936          1  11007.0
NaT         43.995888          2  15214.0
2008-04-08  43.373773          1  11333.0
2008-04-10  43.154738          5  15677.0
2008-04-11  42.921659          2  10792.0
Dataframe preprocesado (posiciones 0:10):
               precio  ubicación   clicks
2008-04-01  43.155647        2.0  18784.0
2008-04-02  43.079056        1.0  24738.0
2008-04-03  43.842609        2.0  15209.0
2008-04-04  43.382794        2.0  14320.0
2008-04-05  43.941176        1.0  11974.0
2008-04-06  44.403936        1.0  11007.0
2008-04-07  43.995888        2.0  15214.0
2008-04-08  43.373773        1.0  11333.0
2008-04-09        NaN        NaN      

Y vemos que este último paso del pre-procesamiento ha generado 3 observaciones *NaN* en la fecha correspondiente a 2008-04-09. Estas tres observaciones faltantes tendremos que manejarlas usando alguna de las técnicas que veremos en las próximas lecciones.